In [3]:
import google.generativeai as genai
import pandas as pd
from typing_extensions import TypedDict
import json

class Entity(TypedDict):
    """
    An entity details in the knowledge graph. The entity detials consist of name, class and description.

    Attributes:
    -----------
    name : str
        The human-readable name of the entity (e.g., Person, Product, etc.)..
    class : str
        The classification or type of the entity (e.g., Person, Product, etc.).
    description : str
        A brief description of the entity.
    """
    entity_name: str
    entity_class: str
    description: str

class Relation(TypedDict):
    """
    A class to represent a relationship (triple) in the knowledge graph.

    Attributes:
    -----------
    head : Entity
        The subject or head entity in the relationship.
    predicate : str
        The relationship or predicate that links the head and tail (e.g., Person, Product, etc.).
    tail : Entity
        The object or tail entity in the relationship.
    """
    head: Entity
    predicate: str
    tail: Entity
    
def add_to_database(
    entities: list[Entity],
    triples: list[Relation],
):
    pass


import time
from functools import wraps

def retry(n):
    """
    A decorator to retry a function n times upon failure, and print the number of attempts made.
    
    Parameters:
    -----------
    n : int
        The number of retry attempts.
    
    Returns:
    --------
    Callable
        A decorator that retries the decorated function up to n times.
    """
    def decorator_retry(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            attempts = 0
            while attempts < n:
                try:
                    attempts += 1
                    # Pass the attempt count to the decorated function
                    result = func(*args, attempt_number=attempts, **kwargs)
                    print(f"Success on attempt {attempts}")
                    return result
                except Exception as e:
                    print(f"Attempt {attempts} failed: {e}")
                    if attempts == n:
                        raise  # Re-raise the exception if it's the last attempt
                    # time.sleep(2)  # Optional delay between retries
        return wrapper
    return decorator_retry

def dp_text_split(text, target):
    # Split the text into lines
    lines = text.split('\n')
    line_lengths = [len(line) for line in lines]
    n = len(lines)

    # Initialize DP table and splits
    dp = [float('inf')] * (n + 1)
    dp[0] = 0  # No error at the start
    splits = [-1] * (n + 1)

    # Dynamic programming logic to calculate the minimum error
    for i in range(1, n + 1):
        total_length = 0
        for j in range(i, 0, -1):
            total_length += line_lengths[j - 1] + 1  # +1 to account for newline

            if total_length - 1 > target and j != i:  # Allow overshoot on the last line
                break

            error = abs(total_length - 1 - target)
            if dp[i] > dp[j - 1] + error:
                dp[i] = dp[j - 1] + error
                splits[i] = j - 1

    # Backtracking to get the chunks
    chunks = []
    i = len(splits) - 1
    while i > 0:
        j = splits[i]
        chunk = '\n'.join(lines[j:i])
        if chunk.strip():  # Avoid adding empty chunks
            chunks.append(chunk)
        i = j

    chunks.reverse()

    # Return the total error and the chunks
    total_error = dp[n]
    return chunks, total_error

@retry(5)
def extract_chunk(chunk, context, model, chunk_id, attempt_number):
    """
    Extracts knowledge graph triples (subject, predicate, object) from Wikipedia text using the Gemini API.

    Parameters:
    -----------
    chunk : str
        The input text from which to extract the knowledge graph.

    Returns:
    --------
    tuple (pd.DataFrame, pd.DataFrame)
        - A pandas DataFrame containing the extracted knowledge graph triples.
        - A pandas DataFrame containing the extracted entities.
    """
    
    
    # Generate the content
    result = model.generate_content(f"""
    # Context
    {context}
    
    # Input Wikipedia Text:
    {chunk}
    
    # Instruction:
    Please extract the entities and relations from the following Wikipedia text to the database.
    Ensure that the entities and relationships are accurate.
    """)
    
    # Parse the function call response
    fc = result.candidates[0].content.parts[0].function_call
    data = type(fc).to_dict(fc)

    # Process entities
    entities_data = data['args']['entities']
    df_entity = pd.DataFrame(entities_data)

    # Process triples
    triples_data = []
    for triple in data['args']['triples']:
        triples_data.append({
            'predicate': triple['predicate'],
            'head': triple['head']['entity_name'],
            'tail': triple['tail']['entity_name']
        })

    df_triple = pd.DataFrame(triples_data)
    df_entity['chunk_id']=chunk_id
    df_triple['chunk_id']=chunk_id
    df_entity['attemp']=attempt_number
    df_triple['attemp']=attempt_number

    return df_triple, df_entity

def extract(wiki_text, context, model, chunk_size=500):
    chunks,_ = dp_text_split(wiki_text, target=chunk_size)
    all_triple=[]
    all_entity=[]
    for i, chunk in enumerate(chunks, 1):
        print(f"Chunk {i}: ({len(chunk)} characters)\n{chunk}\n")
        # Call the function
        df_triple, df_entity = extract_chunk(chunk, context, model, i)
        
        # Display the results
        # print('### Triples')
        # print(df_triple.to_string())
        # print('### Entity')
        # print(df_entity.to_string())
        all_entity.append(df_entity)
        all_triple.append(df_triple)
    df_all_triple = pd.concat(all_triple, ignore_index=True)
    df_all_entity = pd.concat(all_entity, ignore_index=True)
    return df_all_triple, df_all_entity

###############################

In [4]:
# pip install python-dotenv
import os
from dotenv import load_dotenv
load_dotenv()

import os
import google.generativeai as genai
import pandas as pd

# Set up the API key outside the function
genai.configure(api_key=os.getenv("GEMINI_API_KEY"))
# Set up the model and tools (ensure 'add_to_database' is accessible)
model = genai.GenerativeModel(
    model_name='models/gemini-1.5-pro-latest',
    tools=[add_to_database]
)

wiki_text = """
The Liberty Bell Ruby is a sculpture crafted from the world's largest mined ruby,[1] discovered in East Africa in the 1950s.[2] 
It weighs four pounds, is eight and a half thousand carats (8,500), and is sculpted into a miniature form of the Liberty Bell. 
It has 50 diamonds set in it and is valued at $2 million.

The ruby was created in 1976 for Beverly Hills-based Kazanjian Brothers jewelry company by sculptor Alfonso de Vivanco for the United States Bicentennial.[3]
It was made in the same spirit as sapphire busts of presidents that the jeweler's charitable foundation presented to the White House when Dwight D. Eisenhower was president.
"""  
df_all_triple, df_all_entity = extract(wiki_text, {'main topic':'The Liberty Bell Ruby '}, model)
df_all_triple.info()
df_all_triple

Chunk 1: (474 characters)

The Liberty Bell Ruby is a sculpture crafted from the world's largest mined ruby,[1] discovered in East Africa in the 1950s.[2] 
It weighs four pounds, is eight and a half thousand carats (8,500), and is sculpted into a miniature form of the Liberty Bell. 
It has 50 diamonds set in it and is valued at $2 million.

The ruby was created in 1976 for Beverly Hills-based Kazanjian Brothers jewelry company by sculptor Alfonso de Vivanco for the United States Bicentennial.[3]

Attempt 1 failed: 500 An internal error has occurred. Please retry or report in https://developers.generativeai.google/guide/troubleshooting
Attempt 2 failed: 500 An internal error has occurred. Please retry or report in https://developers.generativeai.google/guide/troubleshooting
Attempt 3 failed: 500 An internal error has occurred. Please retry or report in https://developers.generativeai.google/guide/troubleshooting
Success on attempt 4
Chunk 2: (174 characters)
It was made in the same spir

,predicate,head,tail,chunk_id,attemp
0,sculpted from,Liberty Bell Ruby,World\'s largest mined ruby,1,4
1,discovered in,World\'s largest mined ruby,East Africa,1,4
2,discovered in,World\'s largest mined ruby,1950s,1,4
3,sculpted into,Liberty Bell Ruby,Liberty Bell,1,4
4,created by,Liberty Bell Ruby,Alfonso de Vivanco,1,4
5,created for,Liberty Bell Ruby,Kazanjian Brothers,1,4
6,created for,Liberty Bell Ruby,United States Bicentennial,1,4
7,inspired by,The Liberty Bell Ruby,Sapphire busts of presidents,2,1
8,donated,Jeweler's charitable foundation,Sapphire busts of presidents,2,1
9,presented to,Sapphire busts of presidents,White House,2,1


In [5]:
df_all_entity.info()
df_all_entity

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13 entries, 0 to 12
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   description   7 non-null      object
 1   entity_class  13 non-null     object
 2   entity_name   13 non-null     object
 3   chunk_id      13 non-null     int64 
 4   attemp        13 non-null     int64 
dtypes: int64(2), object(3)
memory usage: 648.0+ bytes


,description,entity_class,entity_name,chunk_id,attemp
0,A sculpture crafted from the world\'s largest ...,Sculpture,Liberty Bell Ruby,1,4
1,A ruby discovered in East Africa in the 1950s.,Ruby,World\'s largest mined ruby,1,4
2,NaN,Location,East Africa,1,4
3,NaN,Time Period,1950s,1,4
4,NaN,Monument,Liberty Bell,1,4
5,NaN,Jewelry Company,Kazanjian Brothers,1,4
6,NaN,Sculptor,Alfonso de Vivanco,1,4
7,NaN,Event,United States Bicentennial,1,4
8,,Gemstone,The Liberty Bell Ruby,2,1
9,,Artwork,Sapphire busts of presidents,2,1


In [6]:
wiki_text='''The HeartBeat Monitor Model H200 by CardioLife is designed to continuously monitor patients' heart rates. 
It features wireless connectivity as one of its key characteristics. 
Recently, the device encountered a power failure breakdown caused by a battery defect, 
which was managed by performing a battery replacement repair.'''
df_all_triple, df_all_entity = extract(wiki_text, {'main topic':'HeartBeat Monitor Model H200'}, model)
df_all_triple.info()
df_all_triple

Chunk 1: (326 characters)
The HeartBeat Monitor Model H200 by CardioLife is designed to continuously monitor patients' heart rates. 
It features wireless connectivity as one of its key characteristics. 
Recently, the device encountered a power failure breakdown caused by a battery defect, 
which was managed by performing a battery replacement repair.

Success on attempt 1
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   predicate  5 non-null      object
 1   head       5 non-null      object
 2   tail       5 non-null      object
 3   chunk_id   5 non-null      int64 
 4   attemp     5 non-null      int64 
dtypes: int64(2), object(3)
memory usage: 328.0+ bytes


,predicate,head,tail,chunk_id,attemp
0,manufactured by,HeartBeat Monitor Model H200,CardioLife,1,1
1,has feature,HeartBeat Monitor Model H200,wireless connectivity,1,1
2,encountered,HeartBeat Monitor Model H200,power failure breakdown,1,1
3,caused by,power failure breakdown,battery defect,1,1
4,managed by,power failure breakdown,battery replacement repair,1,1


In [7]:
df_all_entity.info()
df_all_entity

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   description   6 non-null      object
 1   entity_name   6 non-null      object
 2   entity_class  6 non-null      object
 3   chunk_id      6 non-null      int64 
 4   attemp        6 non-null      int64 
dtypes: int64(2), object(3)
memory usage: 368.0+ bytes


,description,entity_name,entity_class,chunk_id,attemp
0,A device designed to continuously monitor pati...,HeartBeat Monitor Model H200,Device,1,1
1,The company that manufactures the HeartBeat Mo...,CardioLife,Company,1,1
2,A feature of the HeartBeat Monitor Model H200.,wireless connectivity,Feature,1,1
3,A problem encountered by the HeartBeat Monitor...,power failure breakdown,Problem,1,1
4,The cause of the power failure breakdown in th...,battery defect,Defect,1,1
5,The solution to the power failure breakdown in...,battery replacement repair,Solution,1,1


In [8]:
wiki_text='''
"สว." จี้รัฐแก้ปัญหาแชร์ลูกโซ่ โดยเฉพาะ "ดิไอคอน" หวั่นจับ "บอส" แล้วจบ แนะตรวจสอบการโอน คริปโต-การเสียภาษี แก้กม.ล้าสมัย เสนอออก พ.ร.ก.แก้ปัญหา ด้าน "จุลพันธ์" รับลูก เตรียมอุดช่องโหว่ ขยายถึงแม่ข่ายกลาง ล่าง เพิ่มโทษ ไม่ให้คดีหมดอายุความ พร้อมออกพ.ร.ก. คาดใช้เวลา 2-3 เดือนเสร็จ เผย 18 บอส บางคนไม่ยื่นภาษี 3-5 ปี ยันไม่มีมวยล้ม เชื่อคลิปเสียงเทวดา ถึงมือตำรวจแล้ว
          ในการประชุมวุฒิสภา วานนี้ (21ต.ค.) นต.วุฒิพงศ์ พงศ์สุวรรณ สว. กระทู้สดด้วยวาจา เรื่องการแก้ไขปัญหาขายตรงแบบแชร์ลูกโซ่ ที่กระทบกับเศรษฐกิจ ว่า กฎหมาย ที่ว่าด้วยเรื่องแชร์ลูกโซ่ ของเราอาจะออกมาเมื่อ 20 กว่าปีที่แล้ว ไม่ทันยุคทันสมัยกับยุคปัจจุบัน
"สิ่งที่เกิดขึ้นไม่ใช่ว่าบอสทั้งหลายโดนจับไปแล้วจะจบ เพราะยังมีอีกหลายต่อหลายบอส ในคดีดิไอคอน และคดีอื่นๆ ที่ตอนนี้กำลังดำเนินการอยู่ ทั้งนี้ เกิดจากประเทศไทยอยู่ในบ่วงโซ่ความโลภ โกรธ หลง ทำให้คนเป็นเยื่อ ผมก็เคยเป็นอยากรวย อยากมีเงิน โดยไม่ต้องทำงาน หรือทำงานน้อยๆ ได้ตังค์เยอะๆ ซึ่งคนไทยส่วนใหญ่อยากรวย เมื่อมีคนที่น่าเชื่อถือมาเล่าเรื่อง เช่น พิธีกรดังๆ เซเลปดังๆ ก็คล้อยตาม ตกเป็นเหยื่อ"
          นอกจากนี้ ยังมีเรื่องของการใช้เงินนอกระบบ หรือ เงิน ที่เป็นคริปโตเคอร์เรนซี ซึ่งเป็นเรื่องที่เป็นอันตรายอย่างยิ่งต่อสังคมไทย คือ การโอนไปต่างประเทศสามารถโอนได้อย่างรวดเร็ว ซึ่งในคดี ดิไอคอน มีคนตรวจสอบแล้วพบว่า มีการโอนไป 8 พันล้านบาท ใช้เวลาไม่เกิน 10 นาที
          ดังนั้น รัฐบาลควรออกพระราชกำหนด (พ.ร.ก.) มาดำเนินการ เพราะเรื่องนี้เป็นเรื่องฉุกเฉิน มีความจำเป็นเร่งด่วน และไม่ต้องกังวลทั้ง สส. และ สว. ก็ให้ความร่วมมือ เพราะทุกคนรู้ว่า เรื่องนี้จำเป็นเร่งด่วน รวมทั้งแก้ไขกฎหมายที่ล้าสมัยด้วย
          สิ่งที่เกิดขึ้นมีคนร้องเรียนตั้งแต่ปี 62 แล้ว มีการแจ้งดำเนินคดี แต่คดีก็หายไป ไม่รู้ว่าเทวดาที่ไหนยับยั้งเอาไว้ ตนคิดว่า เทวดาก็สำคัญ ไม่เช่นนั้นทำไม่ได้ถึงขนาดนี้ และกรมสรรพกร ก็แจ้งมาว่า บอส หลายคนไม่ได้เสียภาษี ตามที่ควรจะเป็น หรืออาจจะไม่มีการเสียภาษีเลย ก็ได้ สิ่งเหล่านี้เป็นสิ่งที่ รมช.คลัง ต้องไปติดตามด้วย รวมถึงรถหรู ที่เป็นการล่อลวงประชาชน ขนาดพระบางรูป ยังไปเทศนาเชิงนิยมชมชื่นด้วย ขอถามว่า เป็นกิจของสงฆ์หรือไม่
          ด้านนายจุลพันธ์ อมรวิวัฒน์ รมช.คลัง ชี้แจงว่า ครม.โดยนายกรัฐมนตรี ให้ความสนใจเรื่องนี้อย่างมาก และได้ให้หน่วยงานที่เกี่ยวข้อง ไปดำเนินการ ทั้ง สตช.ไปดำเนินการเรื่องคดีความ กระทรวงยุติธรรม ดูเรื่องขั้นตอนต่างๆ สคบ.ดูเรื่องการคุ้มครองผู้บริโภค และกระทรวงการคลัง ดูเรื่องการปรับแก้ขั้นตอนกฎหมายต่างๆ ที่มีความจำเป็น ซึ่งในคดี ดิไอคอน นั้น ทั้งหมดอยู่ในข่ายที่ สตช.จะดำเนินการต่อไป ซึ่งทางหน่วยงานที่เกี่ยวข้องก็จะดำเนินการนำทรัพย์สินเหล่านั้น มาเข้าสู่กระบวนการยุติธรรมต่อไป
          อย่างไรก็ตาม ต้องยอมรับว่า เรื่องนี้เกี่ยวกับ พ.ร.บ.หลายฉบับ ทั้ง พ.ร.บ.เกี่ยวกับการขายตรง และตลาดแบบตรง พ.ศ.2545, พ.ร.บ.การกู้ยืมเงินที่เป็นการฉ้อโกงประชาชนปี 2527 หรือ พ.ร.บ.แม่ชม้อย, พ.ร.บ.คุ้มครองผู้บริโภค 2522, ประมวลกฎหมายอาญา ปี 2499, พ.ร.บ.ว่าด้วยการทำผิดเกี่ยวกับคอมพิวเตอร์ ปี 2560, พ.ร.บ. การป้องกันและปราบปรามการฟอกเงินปี 2542
          ดังนั้น ถึงวาระที่เราต้องมาพิจารณา ทบทวน ซึ่งกระทรวงการคลัง ไม่ได้นิ่งนอนใจ โดยสำนักงานเศรษฐกิจการคลัง(สศค.) เป็นผู้ที่ยกร่างกฎหมายหลายฉบับในอดีต แต่ต้องยอมรับว่าในอดีตที่ผ่านมา เมื่อยกร่างแล้วออกเป็น พ.ร.ก. ไม่มีผู้ที่ดูแลอย่างชัดเจน สุดท้ายก็มอบหมายให้ผู้ที่รักษาการตามกฎหมาย คือ รมว.คลัง และรมว.มหาดไทย ดูแลจนถึงปัจจุบัน และ กลไกที่ผ่านมา ก็จะต้องรอให้มีผู้ร้อง และ สศค. จะรวบรวมเพื่อนำไปดำเนินคดีกับหน่วยงานที่เกี่ยวข้องต่อไป ซึ่งต้องยอมรับความจริงว่า สศค. ไม่ได้เป็นหน่วยงานแรก ที่ประชาชนจะนึกถึง การร้องเรียนเรื่องต่างๆ จึงไป ที่ สตช.และ ดีเอสไอ เป็นหลัก
          ทั้งนี้ จากการหารือในเบื้องต้น จะต้องมีการแก้ไข พ.ร.ก.ที่ออกมาตั้งแต่ปี 2527 ในหลายๆประเด็น ซึ่งขณะนี้อยู่ระหว่างยกร่าง ยังไม่ได้ลงในรายละเอียด และกลไกต้องให้เป็นไปตาม ขั้นตอน สุดท้ายต้องเข้ามาสู่กระบวนการตรวจร่างโดยคณะกรรมการกฤษฎีกาเข้า ครม.แล้วประกาศใช้ จากนั้นถึงจะเข้าสู่สภาฯ และวุฒิสภา เพื่ออนุมัติต่อไป เชื่อว่าจะใช้เวลาไม่เกิน 2-3 เดือน ก็จะดำเนินการแล้วเสร็จ เพราะเรื่องนี้ถือว่ามีความจำเป็นเร่งด่วน
          สำหรับประเด็นหลัก ที่มีการตั้งโจทย์ไว้ ในการแก้ไข พ.ร.บ.ปี 2527 คือ เรื่องของบทบัญญัติโดยให้หน่วยงานที่สามารถเอาผิดไปถึงแม่ข่ายได้ ซึ่งกฎหมายที่ใช้อยู่ยังมีช่องโหว่ ไม่สามารถเข้าผิดแม่ข่าย ถึงระดับกลาง ระดับล่างได้ จึงต้องมีการแก้ไข และต้องมีการเพิ่มโทษ ที่บางส่วนยังไม่ได้สัดส่วนกับความเสียหายที่เกิดขึ้น ยังเป็นหลักแสน หลักล้านบาท ขณะที่ความเสียหายที่เกิดขึ้นกับสังคม อย่างตัวเลขในกรณี ดิไอคอน ณ ปัจจุบัน จำนวน 1.6 พันล้านบาทแล้ว และอาจจะขึ้นไปเรื่อยๆ ในกรณีที่มีคน เข้ามาแจ้งความคดีเพิ่มเติม รวมถึงต้องปรับแก้เรื่องของอายุความ เมื่อคดีเข้าสู่อายุความแล้ว กฎหมายของ ป.ป.ง. ก็ดำเนินการต่อทันทีเกี่ยวกับอายัดทรัพย์ และทรัพย์สินต่างๆ
          อย่างไรก็ตาม ปัญหาที่เกิดขึ้นใน พ.ร.บ. ที่ใช้อยู่ในปัจจุบัน คือ เรื่องอายุความ ในกรณีที่ ผู้ถูกกล่าวหาหลบหนีก็ทำให้อายุความขาดได้ เราจึงต้องแก้ให้อายุความ หยุดลงในกรณีที่ผู้ต้องหาหลบหนี กรณีนี้จะเป็นการป้องกันไม่ให้คดีขาดอายุความ เพื่อสามารถดำเนินคดีได้ถึงที่สุด
          เรื่องปรับเปลี่ยนผู้รักษาการตามกฎหมาย เนื่องจาก สศค.เป็นหน่วยงานมีอำนาจในการปฏิบัติ เป็นหน่วยงานในเชิงนโยบาย ซึ่งมองว่ากลไกที่จะมีประโยชน์สูงสุด คือ ต้องให้ผู้ที่ถือกฎหมายฉบับนี้ เป็นส่วนงานที่เกี่ยวข้องกับกระทรวงยุติธรรม เช่น ดีเอสไอ เป็นต้น ที่จะเข้ามาเป็นผู้ถือกฎหมาย เพื่อให้การดำเนินคดีอยู่ในมือผู้ปฏิบัติ และสามารถดำเนินการในรูปแบบวันสต็อป ให้กับประชาชนที่เดือดร้อนได้
          สุดท้ายคือ การปรับแก้ที่สำคัญ เพื่อให้ทันกับสถานการณ์ปัจจุบัน ต้องยอมรับว่ารูปแบบในการฉ้อโกง และแชร์ต่างๆ ปรับเปลี่ยนไปมาก และมีรูปแบบที่หลากหลาย ซึ่งจะไม่หยุดอยู่แค่นี้ เพราะฉะนั้นตัวกฎหมายต้องมีความยืดหยุ่น ปรับตัวให้ทันกับผู้ที่กระทำความผิดในเทคนิค และกลเม็ดใหม่ๆ ทำให้ภาครัฐมีความจำเป็นต้องปรับตัว ซึ่งกฎหมายที่จะปรับแก้ภายใน 2-3 เดือนนี้ เพื่อปิดช่องโหว่ต่างๆ หวังว่าการแก้ไขที่จะเกิดขึ้นในเร็ววัน จะสามารถป้องกันการฉ้อโกงในประเภทต่างๆได้ ต่อไป
          ทั้งนี้ นต.วุฒิพงศ์ ถามย้ำว่า ของบางอย่างที่ต้องการความรวดเร็ว สามารถเสนอเป็น พ.ร.ก. ได้ ในขณะที่ถ้าเป็น พ.ร.บ. ต้องเสนอตามขั้นตอน อาจจะต้องใช้เวลาไม่น่าต่ำกว่า 5-6 เดือน ดังนั้น สิ่งที่จะทำแทนประชาชนได้คือการออก พ.ร.ก. เพื่อใช้แก้ไขปัญหา โดยเร็ว
          นายจุลพันธ์ ชี้แจงว่า เวลาที่ภาครัฐจะดำเนินการทางด้านกฎหมาย ที่เกี่ยวกับการเงิน การคลัง โดยมากจะเลือกช่องทาง พ.ร.ก.อยู่แล้ว เพราะกรณีนี้เข้ากรอบของการออกเป็น พ.ร.ก. ค่อนข้างมาก ส่วนเรื่องภาษี ยืนยันว่า ตรวจสอบ บอส 18 คน ย้อนหลังเรียบร้อย โดยมีบางรายไม่ได้ยื่นแบบภาษี ระหว่าง 3-5 ปี ซึ่งยังไม่ใช่จุดสิ้นสุด จะต้องตรวจสอบว่า หลบเลี่ยงอย่างไร ขอว่าไม่ต้องกังวล.
'''
df_all_triple, df_all_entity = extract(wiki_text, {'main topic':'ข่าวหุ้น'}, model,100)
df_all_triple.info()
df_all_triple

Chunk 1: (366 characters)
"สว." จี้รัฐแก้ปัญหาแชร์ลูกโซ่ โดยเฉพาะ "ดิไอคอน" หวั่นจับ "บอส" แล้วจบ แนะตรวจสอบการโอน คริปโต-การเสียภาษี แก้กม.ล้าสมัย เสนอออก พ.ร.ก.แก้ปัญหา ด้าน "จุลพันธ์" รับลูก เตรียมอุดช่องโหว่ ขยายถึงแม่ข่ายกลาง ล่าง เพิ่มโทษ ไม่ให้คดีหมดอายุความ พร้อมออกพ.ร.ก. คาดใช้เวลา 2-3 เดือนเสร็จ เผย 18 บอส บางคนไม่ยื่นภาษี 3-5 ปี ยันไม่มีมวยล้ม เชื่อคลิปเสียงเทวดา ถึงมือตำรวจแล้ว

Attempt 1 failed: 500 An internal error has occurred. Please retry or report in https://developers.generativeai.google/guide/troubleshooting
Attempt 2 failed: 500 An internal error has occurred. Please retry or report in https://developers.generativeai.google/guide/troubleshooting
Success on attempt 3
Chunk 2: (254 characters)
          ในการประชุมวุฒิสภา วานนี้ (21ต.ค.) นต.วุฒิพงศ์ พงศ์สุวรรณ สว. กระทู้สดด้วยวาจา เรื่องการแก้ไขปัญหาขายตรงแบบแชร์ลูกโซ่ ที่กระทบกับเศรษฐกิจ ว่า กฎหมาย ที่ว่าด้วยเรื่องแชร์ลูกโซ่ ของเราอาจะออกมาเมื่อ 20 กว่าปีที่แล้ว ไม่ทันยุคทันสมัยกับยุคปัจจุบัน

Attempt 1 failed: 500

InternalServerError: 500 An internal error has occurred. Please retry or report in https://developers.generativeai.google/guide/troubleshooting

In [ ]:
df_all_entity.info()
df_all_entity

In [6]:
import pandas as pd
df_text=pd.read_parquet('./staging/text/transformed.parquet').dropna(subset=['len_text'])
df_text.info()
df_text.head()

<class 'pandas.core.frame.DataFrame'>
Index: 682 entries, 0 to 1370
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   wiki_id   682 non-null    object 
 1   url       682 non-null    object 
 2   page      682 non-null    object 
 3   len_text  682 non-null    float64
dtypes: float64(1), object(3)
memory usage: 26.6+ KB


,wiki_id,url,page,len_text
0,Q43436,https://en.wikipedia.org/wiki/Pearl,"Pearl\n\nA pearl is a hard, glistening object ...",38359.0
1,Q43088,https://en.wikipedia.org/wiki/Ruby,Ruby\n\nRuby is a pinkish red to blood-red col...,13666.0
2,Q5283,https://en.wikipedia.org/wiki/Diamond,Diamond\n\nDiamond is a solid form of the elem...,61340.0
3,Q573870,https://en.wikipedia.org/wiki/Bi_(jade),Bi (jade)\n\nThe bi (Chinese: 璧) is a type of ...,3694.0
5,Q138979,https://en.wikipedia.org/wiki/Nephrite,"NephriteNephrite is a variety of the calcium, ...",6175.0


In [28]:
# Create the DataFrame
# df = pd.DataFrame({'x1': [1, 2], 'x2': [3, 4]})

def as_series(func):
    def wrapper(*args, **kwargs):
        result = func(*args, **kwargs)
        return pd.Series(result)
    return wrapper
    
# Define the function to compute y1 and y2
@as_series
def f(text,chunk_size):
    chunks, error = dp_text_split(text,chunk_size)
    return  error, len(chunks), chunks

chunk_size= 2000
df1=df_text.copy()
df1['chunks']=df1['page'].apply(lambda x: len(dp_text_split(x,chunk_size)[0]))

# Apply the function to each row and create 'y1' and 'y2' columns
df1[['error','len_chunk', 'chunks']] = df1.apply(lambda row: f(row['page'],chunk_size), axis=1)
df1.info()
df1.head()

<class 'pandas.core.frame.DataFrame'>
Index: 682 entries, 0 to 1370
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   wiki_id    682 non-null    object 
 1   url        682 non-null    object 
 2   page       682 non-null    object 
 3   len_text   682 non-null    float64
 4   chunks     682 non-null    object 
 5   error      682 non-null    int64  
 6   len_chunk  682 non-null    int64  
dtypes: float64(1), int64(2), object(4)
memory usage: 42.6+ KB


,wiki_id,url,page,len_text,chunks,error,len_chunk
0,Q43436,https://en.wikipedia.org/wiki/Pearl,"Pearl\n\nA pearl is a hard, glistening object ...",38359.0,"[Pearl\n\nA pearl is a hard, glistening object...",7663,23
1,Q43088,https://en.wikipedia.org/wiki/Ruby,Ruby\n\nRuby is a pinkish red to blood-red col...,13666.0,[Ruby\n\nRuby is a pinkish red to blood-red co...,4342,9
2,Q5283,https://en.wikipedia.org/wiki/Diamond,Diamond\n\nDiamond is a solid form of the elem...,61340.0,[Diamond\n\nDiamond is a solid form of the ele...,14697,38
3,Q573870,https://en.wikipedia.org/wiki/Bi_(jade),Bi (jade)\n\nThe bi (Chinese: 璧) is a type of ...,3694.0,[Bi (jade)\n\nThe bi (Chinese: 璧) is a type of...,307,2
5,Q138979,https://en.wikipedia.org/wiki/Nephrite,"NephriteNephrite is a variety of the calcium, ...",6175.0,"[NephriteNephrite is a variety of the calcium,...",3829,5


In [32]:
df2 = df1[['wiki_id','url','chunks']].copy()
df2['chunk_id'] = df2['chunks'].apply(lambda x: list(range(1, len(x) + 1)))

df2=df2.explode(['chunks', 'chunk_id']).reset_index(drop=True)
df2['len'] = df2['chunks'].apply(lambda x: len(x))
df2.to_parquet('./data/91_chunk.parquet')
df2.info()
df2

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6169 entries, 0 to 6168
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   wiki_id   6169 non-null   object
 1   url       6169 non-null   object
 2   chunks    6169 non-null   object
 3   chunk_id  6169 non-null   object
 4   len       6169 non-null   int64 
dtypes: int64(1), object(4)
memory usage: 241.1+ KB


,wiki_id,url,chunks,chunk_id,len
0,Q43436,https://en.wikipedia.org/wiki/Pearl,"Pearl\n\nA pearl is a hard, glistening object ...",1,1880
1,Q43436,https://en.wikipedia.org/wiki/Pearl,The scientific name for the family of pearl-be...,2,1592
2,Q43436,https://en.wikipedia.org/wiki/Pearl,The unique luster of pearls depends upon the r...,3,1440
3,Q43436,https://en.wikipedia.org/wiki/Pearl,The mollusk's mantle (protective membrane) dep...,4,1466
4,Q43436,https://en.wikipedia.org/wiki/Pearl,"Typically, the build-up of a natural pearl con...",5,1782
...,...,...,...,...,...
6164,Q483958,"https://en.wikipedia.org/wiki/Okanogan_County,...","In July 2014, the Carlton Complex wildfire bur...",2,1876
6165,Q483958,"https://en.wikipedia.org/wiki/Okanogan_County,...",The median income for a household in the count...,3,1790
6166,Q483958,"https://en.wikipedia.org/wiki/Okanogan_County,...",Okanogan County was once a national bellwether...,4,531
6167,Q7972122,https://en.wikipedia.org/wiki/Washington_Pass,Washington PassWashington Pass (el. 5477 ft./1...,1,1872


In [33]:
df3=pd.read_parquet('./data/91_chunk.parquet')
df3.info()
df3

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6169 entries, 0 to 6168
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   wiki_id   6169 non-null   object
 1   url       6169 non-null   object
 2   chunks    6169 non-null   object
 3   chunk_id  6169 non-null   int64 
 4   len       6169 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 241.1+ KB


,wiki_id,url,chunks,chunk_id,len
0,Q43436,https://en.wikipedia.org/wiki/Pearl,"Pearl\n\nA pearl is a hard, glistening object ...",1,1880
1,Q43436,https://en.wikipedia.org/wiki/Pearl,The scientific name for the family of pearl-be...,2,1592
2,Q43436,https://en.wikipedia.org/wiki/Pearl,The unique luster of pearls depends upon the r...,3,1440
3,Q43436,https://en.wikipedia.org/wiki/Pearl,The mollusk's mantle (protective membrane) dep...,4,1466
4,Q43436,https://en.wikipedia.org/wiki/Pearl,"Typically, the build-up of a natural pearl con...",5,1782
...,...,...,...,...,...
6164,Q483958,"https://en.wikipedia.org/wiki/Okanogan_County,...","In July 2014, the Carlton Complex wildfire bur...",2,1876
6165,Q483958,"https://en.wikipedia.org/wiki/Okanogan_County,...",The median income for a household in the count...,3,1790
6166,Q483958,"https://en.wikipedia.org/wiki/Okanogan_County,...",Okanogan County was once a national bellwether...,4,531
6167,Q7972122,https://en.wikipedia.org/wiki/Washington_Pass,Washington PassWashington Pass (el. 5477 ft./1...,1,1872


In [35]:
len(df3.loc[1,'chunks'])

1592

In [36]:
df3.loc[1,'chunks']

'The scientific name for the family of pearl-bearing oysters, Margaritiferidae comes from the Old Persian word for pearl *margārīta- which is the source of the English name Margaret.[6][7][8]\n\nAll shelled mollusks can, by natural processes, produce some kind of "pearl" when an irritating microscopic object becomes trapped within its mantle folds, but the great majority of these "pearls" are not valued as gemstones. Nacreous pearls, the best-known and most commercially significant, are primarily produced by two groups of molluskan bivalves or clams. A nacreous pearl is made from layers of nacre, by the same living process as is used in the secretion of the mother of pearl which lines the shell.\n\nNatural (or wild) pearls, formed without human intervention, are very rare. Many hundreds of pearl oysters or mussels must be gathered and opened, and thus killed, to find even one wild pearl; for many centuries, this was the only way pearls were obtained, and why pearls fetched such extraor

In [44]:
@as_series
def my_extract(chunk, chunk_id, context):
    df_triple, df_entity = extract_chunk(chunk, context, model, chunk_id)
    # df_triple,df_entity = extract(text, context, model)
    return  df_triple,df_entity

df4=df3.head(1).copy()
# df4['chunks']=df4['page'].apply(lambda x: len(dp_text_split(x,500)[0]))
# Apply the function to each row and create 'y1' and 'y2' columns
df4[['triple','entity']] = df4.apply(lambda row: my_extract(row['chunks'], row['chunk_id'] ,{'article url': row['url']}), axis=1)
df4.info()
df4.head()

Success on attempt 1
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   wiki_id   1 non-null      object
 1   url       1 non-null      object
 2   chunks    1 non-null      object
 3   chunk_id  1 non-null      int64 
 4   len       1 non-null      int64 
 5   triple    1 non-null      object
 6   entity    1 non-null      object
dtypes: int64(2), object(5)
memory usage: 184.0+ bytes


,wiki_id,url,chunks,chunk_id,len,triple,entity
0,Q43436,https://en.wikipedia.org/wiki/Pearl,"Pearl\n\nA pearl is a hard, glistening object ...",1,1880,predicate head ...,desc...


In [46]:
df4.loc[0,'triple']

,predicate,head,tail,chunk_id,attemp
0,is composed of,Pearl,Calcium carbonate,1,1
1,produced by,Pearl,Mollusk,1,1
2,produced by,Pearl,Conulariid,1,1
3,are,Natural pearls,Rare,1,1
4,produced by,Cultured pearls,Pearl oysters,1,1
5,produced by,Cultured pearls,Freshwater mussels,1,1


In [47]:
df4.loc[0,'entity']

,description,entity_name,entity_class,chunk_id,attemp
0,"A hard, glistening object produced within the ...",Pearl,object,1,1
1,A type of animal that produces pearls.,Mollusk,animal,1,1
2,An extinct animal that produced pearls.,Conulariid,animal,1,1
3,A mineral found in pearls.,Aragonite,mineral,1,1
4,A mineral found in pearls.,Calcite,mineral,1,1
5,Pearls that are not perfectly round and smooth.,Baroque pearls,object,1,1
6,Highly valued natural pearls.,Gemstones,object,1,1
7,Pearls that occur spontaneously in the wild.,Natural pearls,object,1,1
8,Pearls that are farmed from pearl oysters and ...,Cultured pearls,object,1,1
9,A type of mollusk that produces pearls.,Pearl oysters,animal,1,1


## note

In [ ]:
from dask.distributed import Client, LocalCluster

# Set up a local Dask cluster
cluster = LocalCluster(n_workers=8)  # You can adjust settings here if needed (e.g., number of workers)
cluster

import dask.dataframe as dd
import pandas as pd
from dask import delayed
from tqdm.auto import tqdm
from dask.distributed import Client, LocalCluster

# Assume e2spo is a function that takes an entity ID and returns a Pandas DataFrame
# def e2spo(entity_id):
    # Your logic to fetch data goes here
    # Return a Pandas DataFrame
    # pass

# Create a delayed version of e2spo
@delayed
def delayed_e2spo(entity_id):
    df = e2spo(entity_id)
    df['source'] = entity_id
    return df

# Use context manager to create a Dask client
with Client(cluster) as client:
    
    # List to hold delayed tasks
    delayed_tasks = []
    
    # Use tqdm for progress bar
    pbar = tqdm(entity_list)
    for e in pbar:
        pbar.set_description(f'Query entity: {e:<15}')
        delayed_tasks.append(delayed_e2spo(e))
    
    # Compute the tasks in parallel using the Dask cluster
    list_df = delayed(delayed_tasks).compute()

    # Concatenate the DataFrames row-wise
    df = pd.concat(list_df, axis=0, ignore_index=True)

    # Convert Pandas DataFrame to Dask DataFrame for writing to Parquet
    ddf = dd.from_pandas(df, npartitions=10)

    # Write the Dask DataFrame to partitioned Parquet
    parquet_dir = 'parquet_dir/'
    ddf.to_parquet(parquet_dir, partition_on=['source'], engine='pyarrow')

    # Display some information about the Dask DataFrame
    print("Partitioned Dask DataFrame written to Parquet.")
